### Patients
patient_id, name,age, arrival_date, departure_date, service, satisfaction
### Services Weekly
week, month, service, available_beds,patients_request, patients_admitted,
patients_refused, patient_satisfaction, staff_morale,event
### Staff Schedule
week, staff_id, staff_name, role, service, present
### Staff
staff_id,staff_name,role,service

In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import os

pio.renderers.default = "notebook_connected"
# pio.renderers.default = "iframe"

# Do not truncate tables
pd.set_option('display.max_columns', None)

### 0. Data Loading

In [18]:
PATIENTS_CSV = '../data/patients.csv'
SERVICES_CSV = '../data/services_weekly.csv'
STAFF_CSV = '../data/staff.csv'
SCHEDULE_CSV = '../data/staff_schedule.csv'

df_patients = pd.read_csv(PATIENTS_CSV, delimiter=',', low_memory=False)
df_staff = pd.read_csv(STAFF_CSV, delimiter=',', low_memory=False)
df_staff_schedule = pd.read_csv(SCHEDULE_CSV, delimiter=',', low_memory=False)
df_services_weekly = pd.read_csv(SERVICES_CSV, delimiter=',', low_memory=False)

dfs = [df_patients, df_services_weekly, df_staff, df_staff_schedule]

### 1. Data Types

In [19]:
df_patients['arrival_date'] = pd.to_datetime(df_patients['arrival_date'], errors='coerce')
df_patients['departure_date'] = pd.to_datetime(df_patients['departure_date'], errors='coerce')

# Convert int columns to numeric
df_patients['age'] = pd.to_numeric(df_patients['age'], errors='coerce')
df_patients['satisfaction'] = pd.to_numeric(df_patients['satisfaction'], errors='coerce')

df_staff_schedule['week'] = pd.to_numeric(df_staff_schedule['week'], errors='coerce')
df_staff_schedule['present'] = pd.to_numeric(df_staff_schedule['present'], errors='coerce')

numeric_cols = [
    'week', 'month', 'available_beds', 'patients_request', 'patients_admitted',
    'patients_refused', 'patient_satisfaction', 'staff_morale'
]
for col in numeric_cols:
    df_services_weekly[col] = pd.to_numeric(df_services_weekly[col], errors='coerce')

### 2. Check for empty values

In [20]:
def find_empty_values(df: pd.DataFrame) -> dict[str, int]:
    empty_values_by_column: dict[str, int] = dict()
    for column in df.columns:
        empty_values_by_column[column] = df[column].isnull().sum()
    return empty_values_by_column

def print_empty_values(empty_values_by_column: dict[str, int]) -> None:
    print("Empty values by column:")
    for column, empty_count in empty_values_by_column.items():
        print(f"{column}: {empty_count}")

In [21]:
exist_empty = False
for df in dfs:
    empty_values = find_empty_values(df)
    if any(count > 0 for count in empty_values.values()):
        exist_empty = True
        print_empty_values(empty_values)

if not exist_empty:
    print("No empty values found in any dataset.")

No empty values found in any dataset.


### 3. Check invalid data

In [22]:
def check_invalid_dates(df: pd.DataFrame, date_columns: list[str]) -> None:
    for column in date_columns:
        invalid_dates = df[pd.to_datetime(df[column], errors='coerce').isna()]
        if not invalid_dates.empty:
            print(f"Invalid dates found in column {column}:\n{invalid_dates}")

def check_negative(df: pd.DataFrame, column: str) -> None:
    negative_values = df[df[column] < 0]
    if not negative_values.empty:
        print(f"Negative values found in column {column}:\n{negative_values}")

def check_out_of_range(df: pd.DataFrame, column: str, min_value: float, max_value: float) -> None:
    out_of_range_values = df[(df[column] < min_value) | (df[column] > max_value)]
    if not out_of_range_values.empty:
        print(f"Out of range values found in column {column}:\n{out_of_range_values}")

def check_id(df: pd.DataFrame, id_column: str, person_type: str) -> None:
    # Patient IDs start with PAT, Staff IDs start with STF
    code = 'PAT' if person_type == 'patient' else 'STF'
    invalid_code_ids = df[~df[id_column].str.startswith(code, na=False)]

    # All IDs are in the form `{code}-{8 digit/letter combination}`
    invalid_format_ids = invalid_code_ids[~invalid_code_ids[id_column].str.match(rf'^{code}-[A-Za-z0-9]{{8}}$', na=False)]

    if not invalid_format_ids.empty:
        print(f"Invalid {person_type} IDs found in column {id_column}:\n{invalid_format_ids}")

In [23]:
# Check for negative ages in patients dataset
check_negative(df_patients, 'age')
# Check for invalid dates in patients dataset
check_invalid_dates(df_patients, ['arrival_date', 'departure_date'])
# Check weeks in range 1-52 in services_weekly dataset
check_out_of_range(df_services_weekly, 'week', 1, 52)
# Check months in range 1-12 in services_weekly dataset
check_out_of_range(df_services_weekly, 'month', 1, 12)
# Check patient IDs in patients dataset
check_id(df_patients, 'patient_id', 'patient')
# Check staff IDs in staff dataset
check_id(df_staff, 'staff_id', 'staff')

### 4. Check for duplicates

In [24]:
def check_duplicates(df: pd.DataFrame, df_name: str) -> None:
    duplicates = df[df.duplicated()]
    if not duplicates.empty:
        print(f"Duplicate rows found in {df_name}:\n{duplicates}")

for index, df in enumerate(dfs):
    check_duplicates(df, f"dataset_{index}")

### 5. Add useful fields

In [25]:
df_patients['length_of_stay'] = (pd.to_datetime(df_patients['departure_date']) - pd.to_datetime(df_patients['arrival_date'])).dt.days
df_patients['week'] = pd.to_datetime(df_patients['arrival_date']).dt.isocalendar().week
df_patients['month'] = pd.to_datetime(df_patients['arrival_date']).dt.month

staff_presence = df_staff_schedule.groupby(['service', 'week']).agg(
    staff_present_total=('present', 'sum'),
    doctors_present=('role', lambda x: ((x == 'doctor') & (df_staff_schedule.loc[x.index, 'present'] == 1)).sum()),
    nurses_present=('role', lambda x: ((x == 'nurse') & (df_staff_schedule.loc[x.index, 'present'] == 1)).sum()),
    assistants_present=('role', lambda x: ((x == 'nursing_assistant') & (df_staff_schedule.loc[x.index, 'present'] == 1)).sum())
).reset_index()

In [26]:
staff_presence.head(5)

,service,week,staff_present_total,doctors_present,nurses_present,assistants_present
0,ICU,1,31,6,17,8
1,ICU,2,30,6,17,7
2,ICU,3,0,0,0,0
3,ICU,4,29,5,16,8
4,ICU,5,28,5,15,8


In [27]:

# 1. Prepare Data
# Create the 'status' variable to split the data (Demand vs Availability)
df_services_weekly['availability_status'] = df_services_weekly.apply(
    lambda row: 'Shortage' if row['patients_request'] > row['available_beds'] else 'Sufficient', 
    axis=1
)

df_services_weekly['patients_refused_rate'] = df_services_weekly['patients_refused'] / df_services_weekly['patients_request']

# Metrics to analyze on the Radar Chart
metrics = ['patient_satisfaction', 'staff_morale', 'patients_refused_rate']
# Clean names for the chart
metric_labels = ['Satisfaction', 'Staff Morale', 'Refusals']

# Aggregate: Get the average performance for each Service + Status combination
df_agg = df_services_weekly.groupby(['service', 'availability_status'])[metrics].mean().reset_index()

# 2. Normalize Data for Radar Chart
df_agg['patient_satisfaction'] = df_agg['patient_satisfaction'] / 100.0
df_agg['staff_morale'] = df_agg['staff_morale'] / 100.0

for metric in metrics:
    df_agg[metric] = (df_agg[metric]).round(2)

print(df_agg)

            service availability_status  patient_satisfaction  staff_morale  \
0               ICU            Shortage                  0.83          0.72   
1               ICU          Sufficient                  0.81          0.70   
2         emergency            Shortage                  0.78          0.74   
3  general_medicine            Shortage                  0.80          0.73   
4  general_medicine          Sufficient                  0.85          0.72   
5           surgery            Shortage                  0.79          0.77   
6           surgery          Sufficient                  0.80          0.68   

   patients_refused_rate  
0                   0.25  
1                   0.00  
2                   0.77  
3                   0.42  
4                   0.00  
5                   0.31  
6                   0.00  


In [31]:
# 3. Build the Diagram (Faceted Radar Chart)
# Create a 2x2 grid of polar subplots
services = list(df_services_weekly['service'].unique())

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'polar'}]*2]*2, # Define each subplot as a polar type
    subplot_titles=services # Titles for each small multiple
)

# Define colors for the two states
colors = {'Sufficient': '#1f77b4', 'Shortage': '#d62728'}

# Loop through services and add traces
for i, service in enumerate(services):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    for status in ['Sufficient', 'Shortage']:
        # Filter data for this specific polygon
        subset = df_agg[(df_agg['service'] == service) & (df_agg['availability_status'] == status)]
        
        if not subset.empty:
            # Get values and close the loop (repeat first value at the end)
            values = subset[metrics].values.flatten().tolist()
            values += [values[0]] 
            
            theta = metric_labels + [metric_labels[0]]
            
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=theta,
                fill='toself',
                name=status,
                line_color=colors[status],
                opacity=0.6,
                showlegend=(i==1) # Only show legend once
            ), row=row, col=col)

fig.update_layout(
    title_text="<b>Impact of Bed Availability on Hospital Performance</b>"
    "<br><i>Comparison of Satisfaction, Morale and Refusals</i>",
    height=800,
    width=1000,
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    polar2=dict(radialaxis=dict(visible=True, range=[0, 1])),
    polar3=dict(radialaxis=dict(visible=True, range=[0, 1])),
    polar4=dict(radialaxis=dict(visible=True, range=[0, 1])),
    legend=dict(title="Bed Availability", x=1.05, y=1)
)

fig.show()

### Examples (not entirely correct)

In [ ]:
"""
Here are the five visualizations we must generate and verify:
1. Bed Capacity Pressure Analysis (Parallel Coordinates + Small Multiples)
2. Seasonal Patient Load & Satisfaction Trends (Heatmap + Line)
3. Staffing vs Patient Outcomes Diagnostic (SPLOM + Bubble)
4. Staff Allocation Adequacy Review (Stacked Bar + Line, Radar)
5. Operational Behavioral Clustering (k-means clusters, boxplots, cluster timeline)
"""
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.graph_objects as go
import plotly.express as px

# 1.2 Aggregate Patient Metrics by Week & Service
patients_agg = df_patients.groupby(['week', 'service']).agg({
    'length_of_stay': 'mean',
    'satisfaction': 'mean'
}).reset_index().rename(columns={
    'length_of_stay': 'avg_los', 
    'satisfaction': 'avg_patient_sat'
})

# 1.3 Aggregate Staff Presence by Role per Week & Service
# Merge schedule with staff to get roles
schedule_merged = df_staff_schedule.merge(df_staff[['staff_id', 'role']], on='staff_id', how='left')
# Filter for present staff only
schedule_present = schedule_merged[schedule_merged['present'] == 1]

# Pivot to get counts of each role per week/service
staff_pivot = schedule_present.pivot_table(
    index=['week', 'service'], 
    columns='role_y', # using role_y from the merge
    values='staff_id', 
    aggfunc='count',
    fill_value=0
).reset_index()

# Calculate Total Staff
role_cols = [c for c in staff_pivot.columns if c not in ['week', 'service']]
staff_pivot['total_staff'] = staff_pivot[role_cols].sum(axis=1)

# 1.4 Create Master DataFrame (Merge Services + Patients Agg + Staff Agg)
master_df = df_services_weekly.merge(patients_agg, on=['week', 'service'], how='left')
master_df = master_df.merge(staff_pivot, on=['week', 'service'], how='left')
master_df.fillna(0, inplace=True) # Handle weeks with no patient/staff data


# --- Step 2: Visualization Generation (Plotly) ---

# --- Diagram 1: Parallel Coordinates (Bed Demand vs Availability) ---
# Normalizing data for cleaner parallel coordinates
df_norm = master_df.copy()
cols_task1 = ['available_beds', 'patients_request', 'patients_admitted', 'patients_refused']

# Normalize columns
for col in cols_task1:
    df_norm[col] = (df_norm[col] - df_norm[col].min()) / (df_norm[col].max() - df_norm[col].min())

service_map = {name: i for i, name in enumerate(master_df['service'].unique())}
df_norm['service_id'] = df_norm['service'].map(service_map)

fig1 = px.parallel_coordinates(
    df_norm, 
    dimensions=cols_task1,
    color='service_id', 
    labels={col: col.replace('_', ' ').title() for col in cols_task1},
    title="Task 1: Bed Availability vs. Demand (Normalized)"
)

# Update traces to show ranges clearly
fig1.update_traces(dimensions=[
    dict(range=[0,1], label='Available Beds', values=df_norm['available_beds']),
    dict(range=[0,1], label='Requests', values=df_norm['patients_request']),
    dict(range=[0,1], label='Admitted', values=df_norm['patients_admitted']),
    dict(range=[0,1], label='Refused', values=df_norm['patients_refused'])
])
# fig1.write_html("iframe_figures/bed_availability_vs_demand_parallel_coordinates.html")
fig1.show()


# --- Diagram 2a: Heatmap Calendar (Seasonal Patterns) ---
fig2a = px.density_heatmap(
    master_df, 
    x="week", 
    y="service", 
    z="patients_admitted", 
    histfunc="avg",
    title="Task 2a: Seasonal Patient Load (Admissions Heatmap)",
    color_continuous_scale="Viridis",
    labels={'patients_admitted': 'Admissions'}
)
fig2a.update_layout(xaxis_title="Week of Year", yaxis_title="Service")
# fig2a.write_html("iframe_figures/seasonal_patient_load_admissions_heatmap.html")
fig2a.show()


# --- Diagram 2b: SPLOM (Scatter Plot Matrix) ---
# analyzing correlations between Load, Satisfaction, and Morale
fig2b = px.scatter_matrix(
    master_df,
    dimensions=['patients_admitted', 'patient_satisfaction', 'staff_morale'],
    color='service',
    title="Task 2b: Multivariate Correlations (Load vs. Satisfaction vs. Morale)",
    opacity=0.7
)
fig2b.update_traces(diagonal_visible=False)
# fig2b.write_html("iframe_figures/multivariate_correlations_load_vs_satisfaction_vs_morale.html")
fig2b.show()


# --- Diagram 3: Bubble Chart (Staff Presence vs Patient Outcomes) ---
# --- Diagram 3: Staffing vs Patient Outcomes (Refined) ---

# 3a. Bubble Chart (As requested previously)
fig3a = px.scatter(
    master_df, 
    x="total_staff", 
    y="avg_patient_sat",
    size="avg_los", 
    color="service",
    hover_name="service",
    size_max=40,
    title="Task 3a: Staff Presence vs. Patient Satisfaction (Size = Length of Stay)",
    labels={
        "total_staff": "Total Staff Present", 
        "avg_patient_sat": "Avg Satisfaction",
        "avg_los": "Avg LOS (days)"
    }
)
fig3a.show()

# 3b. SPLOM (Scatter Plot Matrix) - NEW REQUEST
# This visualizes pairwise relationships between staffing, load, and outcomes simultaneously.
fig3b = px.scatter_matrix(
    master_df,
    dimensions=["total_staff", "avg_patient_sat", "avg_los", "patients_admitted"],
    color="service",
    title="Task 3b: Multivariate Diagnostic (Staffing, Satisfaction, LOS, Admissions)",
    labels={
        "total_staff": "Staff",
        "avg_patient_sat": "Sat",
        "avg_los": "LOS",
        "patients_admitted": "Admits"
    },
    opacity=0.7
)
fig3b.update_traces(diagonal_visible=False, showupperhalf=False)
fig3b.update_layout(width=1000, height=800)
fig3b.show()


# --- Diagram 4: Stacked Bar + Line (Resource Allocation) ---
# Filter for 'Emergency' service as an example
service_filter = 'emergency'
df_srv = master_df[master_df['service'] == service_filter].sort_values('week')

fig4 = go.Figure()

# Add Stacked Bars for Roles
roles_to_plot = ['doctor', 'nurse', 'nursing_assistant'] 
colors = ['#636EFA', '#EF553B', '#00CC96']

for i, role in enumerate(roles_to_plot):
    # Check if role column exists to avoid errors
    if role in df_srv.columns:
        fig4.add_trace(go.Bar(
            x=df_srv['week'], 
            y=df_srv[role], 
            name=role.capitalize(),
            marker_color=colors[i % len(colors)]
        ))

# Add Line for Patient Load
fig4.add_trace(go.Scatter(
    x=df_srv['week'], 
    y=df_srv['patients_admitted'], 
    name='Patients Admitted',
    mode='lines+markers',
    line=dict(color='black', width=3),
    yaxis='y2'
))

fig4.update_layout(
    title=f"Task 4: Staff Allocation vs Patient Load ({service_filter.capitalize()})",
    xaxis_title="Week",
    yaxis=dict(title="Staff Count"),
    yaxis2=dict(title="Patients Admitted", overlaying='y', side='right'),
    barmode='stack',
    legend=dict(x=1.1, y=1)
)
# fig4.write_html("iframe_figures/staff_allocation_vs_patient_load_emergency.html")
fig4.show()


# --- Diagram 5: Clustering Operational Patterns ---
# Prepare Features
cluster_features = ['patients_admitted', 'staff_morale', 'patient_satisfaction', 'available_beds']
X = master_df[cluster_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Cluster
kmeans = KMeans(n_clusters=3, random_state=42)
master_df['cluster'] = kmeans.fit_predict(X_scaled)
master_df['cluster'] = master_df['cluster'].astype(str) # Convert to string for discrete color mapping

# Boxplot Visualization
fig5 = px.box(
    master_df, 
    x="cluster", 
    y="patients_admitted", 
    color="cluster",
    points="all",
    title="Task 5: Patient Admissions Distribution by Operational Cluster",
    labels={'cluster': 'Cluster ID', 'patients_admitted': 'Patients Admitted'}
)
# fig5.write_html("iframe_figures/patient_admissions_distribution_by_cluster.html")
fig5.show()



# 1. Calculate Centroids (Average values per cluster)
cluster_means = master_df.groupby('cluster')[cluster_features].mean().reset_index()

# 2. Normalize the means (0-1) so they fit nicely on the Radar Chart
# (Without normalization, metrics with large numbers like 'beds' dominate the shape)
scaler = MinMaxScaler()
cluster_means_norm = cluster_means.copy()
cluster_means_norm[cluster_features] = scaler.fit_transform(cluster_means[cluster_features])

# 3. Melt dataframe for Plotly Express (Long Format)
df_radar = cluster_means_norm.melt(
    id_vars='cluster', 
    var_name='Metric', 
    value_name='Normalized_Value'
)

# 4. Generate Radar Chart
fig5 = px.line_polar(
    df_radar, 
    r='Normalized_Value', 
    theta='Metric', 
    color='cluster', 
    line_close=True,
    title="Task 5: Operational Cluster Profiles (Centroids)",
    markers=True,
    category_orders={"cluster": ["0", "1", "2"]} # Ensure consistent color mapping
)

fig5.update_traces(fill='toself', opacity=0.4)
fig5.update_layout(
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 1])
    ),
    legend_title_text='Cluster'
)
fig5.show()